In [ ]:
import argparse
import logging
import os
import sys
import itertools
from itertools import accumulate
from IPython import display

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torch_data
import torch.optim as optim
import matplotlib.pyplot as plt

sys.path.insert(0, "../src")
from collection import Collection

BASE_DIR = os.path.abspath('')

import torch.nn.functional as f


In [2]:
torch.manual_seed(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
# Params
posting_length_to_use = 128

scaling_factor=30

# Fix this
learning_rate = 1e-3
lambda_l2 = 1e-5


In [4]:
torch.set_default_tensor_type(torch.DoubleTensor)


In [5]:
def zigzag_encode (i):
    return (i >> 31) ^ (i << 1)

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, data_list, pos_list):
        self.data = data_list
        self.target = pos_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.target[index]



In [ ]:
import pickle

deltas_list = []
global_max = 0


# Load data
test_collection = Collection("../test_data/test_collection")
for term_id, pl in enumerate(test_collection):
    p = pl[0]
    term_len = len(p)
    if len(p) >= posting_length_to_use:
        model = nn.Sequential(
            nn.Linear(term_len, int(term_len/scaling_factor)),
            nn.ReLU(),
            nn.Linear(int(term_len/scaling_factor), term_len)
        )
        model.to(device)

#         criterion = torch.nn.L1Loss()
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2
#         optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # built-in L2

        X = np.arange(term_len).astype(np.double)
        X = torch.arange(term_len, dtype=torch.double).to(device)
        y = p.astype(np.double)
        y = torch.tensor(y, dtype=torch.float64).to(device)
        for t in itertools.count():
            y_pred = model(X)
            loss = criterion(y_pred,y)
            print("[TERM]: %i, [TERM_LEN]: %i, [EPOCH]: %i, [LOSS]: %.6f" % (term_id, term_len, t, loss.item()))
            if loss < 0.001 or t > 10000:
                deltas = y_pred.detach().numpy().astype(int) - y.numpy().astype(int)
                print(deltas)
                delta_zigzag = [len(deltas)] + list(accumulate([zigzag_encode(int(i))+1 for i in deltas]))            
                global_max = max(delta_zigzag[-1], global_max)
                deltas_list.append(np.array(delta_zigzag, dtype=np.uint32))
#                 print(deltas)
#                 print(max(deltas/y))
#                 print(y)
#                 print(y_pred)
                plt.plot(np.arange(y_pred.size(0)), pl[0], label="Ground truth")
                plt.plot(np.arange(y_pred.size(0)), y_pred.cpu().detach().numpy(), label="Predictions")
                plt.xlabel("Posting list Index")
                plt.ylabel("Doc id")
                plt.title("Predicted vs. Ground truth Doc IDs")
                plt.legend(loc="best")
                plt.show()
#             if (loss < 2):
#                 print(y_pred.detach().numpy().astype(int).tolist())
#                 print(y.numpy().astype(int).tolist())
#                 print(np.count_nonzero(deltas))
                print("[TERM]: %i, [TERM_LEN]: %i, [EPOCH]: %i, [LOSS]: %.6f" % (term_id, term_len, t, loss.item()))
#                 deltas+=1
#                 print(list(accumulate([zigzag_encode(int(i)) for i in deltas])))
                break
#             display.clear_output(wait=True)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if len(deltas_list) > 1:
                break

            
global_max_list = [1, global_max]            
deltas_array = np.concatenate([np.array(global_max_list, dtype=np.uint32)] + deltas_list).ravel()
DELTA_DIR = os.path.join(BASE_DIR, "../deltas")
delta_file = os.path.join(DELTA_DIR, "{}.docs".format("NN"))
print("Saving deltas to: {}".format(delta_file))
with open(delta_file, "wb") as binfile:
    deltas_array.tofile(binfile)

freqs_file = os.path.join(DELTA_DIR, "{}.freqs".format("NN"))
print("Saving freqs to: {}".format(freqs_file))
with open(freqs_file, "wb") as binfile:
    deltas_array[1:-1].tofile(binfile)


In [ ]:
global_max_list = [1, global_max]            
deltas_array = np.concatenate([np.array(global_max_list, dtype=np.uint32)] + deltas_list).ravel()
DELTA_DIR = os.path.join(BASE_DIR, "../deltas")
delta_file = os.path.join(DELTA_DIR, "{}.docs".format("NN"))
print("Saving deltas to: {}".format(delta_file))
with open(delta_file, "wb") as binfile:
    deltas_array.tofile(binfile)


In [ ]:
freqs_file = os.path.join(DELTA_DIR, "{}.freqs".format("NN"))
print("Saving freqs to: {}".format(freqs_file))
with open(freqs_file, "wb") as binfile:
    deltas_array[1:-1].tofile(binfile)


In [ ]:
orig_list = []
orig_max = 0

for term_id, pl in enumerate(test_collection):
    p = pl[0]
    term_len = len(p)
    if len(p) >= posting_length_to_use:
        orig_max = max(p[-1], orig_max)
        orig_list.append(np.append( np.array([len(p)], dtype=np.uint32),  np.array(p, dtype=np.uint32)))
    if len(orig_list) > 1:
            break

orig_max_list = [1, orig_max]            
orig_array = np.concatenate([np.array(orig_max_list, dtype=np.uint32)] + orig_list).ravel()
DELTA_DIR = os.path.join(BASE_DIR, "../deltas")
orig_file = os.path.join(DELTA_DIR, "{}.docs".format("orig"))
print("Saving orig to: {}".format(orig_file))
with open(orig_file, "wb") as binfile:
    orig_array.tofile(binfile)
